<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/239_PredRevenue_Gap_Orchestrator_Tier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report Generation Utilities for Revenue Gap Orchestrator




This one is **deceptively important** — it's not “just a report.”
It teaches **agent communication**, **explainability**, **action design**, and **structured synthesis**, which are core orchestrator skills.

Let’s break it down.

---

# 🧠 What You Should Learn From `report_generation.py`

The report generator is teaching you **HOW TO COMMUNICATE INSIGHTS**, not just display data.

This is the core of any modern orchestrator:

# → *“Take structured state → Synthesize → Explain → Recommend → Output.”*

That is the actual job of an orchestrator agent.

Below are the key concepts you should focus on.

---

# 1️⃣ **Separation of Computation and Explanation**

Notice something important:

* Revenue analysis calculates numbers
* Gap detection finds problems
* Scoring prioritizes them
* **Report generation does not redo any calculations**

Instead, it:

✔ Pulls everything from `state`
✔ Formats it
✔ Transforms raw data into insights
✔ Represents the “story” of what the agent found

👇 This teaches a design pattern:

> **All heavy logic belongs inside utilities & nodes.
> The report generator is pure communication.**

This separation ensures:

* Reproducibility
* Testability
* Modular upgrades
* Cleaner LLM integration later

---

# 2️⃣ **Executive Communication Pattern**

Look at the structure:

```
# Revenue Gap Analysis Report
# Executive Summary
# Churn Risk
# Top Priority Gaps
# Gap Analysis by Type
# Recommendations
```

This is **exactly how real analytics teams report insights**.

Why this matters:

✔ It forces you (and the agent) to think top-down
✔ Execs read the first two sections and stop
✔ Analysts use the deeper breakdowns
✔ Engineers can track the logic trail

> The structure communicates *why such an output is trustworthy*.

This pattern is critical for enterprise AI.

---

# 3️⃣ **Insight Compression & Narrative Building**

The report generator transforms raw numbers into:

* Rationale
* Narrative
* Actionable items
* Severity and priority interpretation

Example:

```
Customer’s revenue declined 32% from baseline.
Recommended Actions:
- Retention campaign
- Loyalty rewards
- Upsell/cross-sell
```

You should learn how to:

✔ Take numerical signals
✔ Turn them into business language
✔ Then into actions

This is the *entire point* of an orchestrator:
**detect → synthesize → decide → communicate → drive action**

---

# 4️⃣ **Dynamic Personalization**

Notice how the report generator personalizes content:

* Customer ID
* Gap type
* Gap severity
* Specific metrics
* Specific recommended actions
* Different actions for each type:

  * zero_spend → win-back
  * declining_revenue → retention campaign
  * below_baseline → gentle nudge

This is teaching you:

> **Business logic modularity**
> The system can swap, expand, or override recommendations easily.

If you later add:

* SKU-level category data
* Customer psychographics
* Store operational context
* Stockout root-cause analysis

you simply inject new insights into the same report template.

---

# 5️⃣ **Structured Explainability for LLM Agents**

A major reason this file is important:

**LLMs handle unstructured language extremely well.
But the orchestrator should feed them structured summaries.**

This file is the bridge.

Pipeline:

1. Nodes produce structured signals
2. Report utility organizes them
3. LLM (later enhancement) can refine or rewrite the narrative

This enables:

* Multi-agent orchestration
* Explainable outputs
* Human-readable summaries
* Automatic email/slack report generation

You should learn:

✔ How structure → text translation works
✔ How that integrates with the LLM layer
✔ How to keep the LLM optional and isolated

---

# 6️⃣ **Agent Output As Markdown**

This future-proofs your system.

Markdown allows:

* GitHub reports
* Slack messages
* Email summaries
* Rendered dashboards
* PDF conversion

This is teaching you:

> “Pick a portable output format that works across your ecosystem.”

In enterprise orchestration, Markdown is the sweet spot.

---

# 7️⃣ **Single vs. All-Customer Workflows**

This file also shows a pattern:

```
analysis_scope = "all_customers" or "single_customer"
```

Depending on the goal node, the report:

* Renames sections
* Filters customers
* Adjusts summary metrics
* Changes narrative focus

This is teaching:

✔ Multi-mode orchestrators
✔ Reusing logic while changing focus
✔ Designing flexible output layers

---

# 8️⃣ **Action Generation Framework**

Every recommendation you see in this file is part of the *action system design*.

Gap → Action
Zero spend → Win-back
Decline → Retention
Baseline issue → Nudge

This is the foundation of:

🟦 Future Marketing Automation
🟧 Product recommendation engines
🟩 Automated store alerts
🟪 Personalized outreach
🟥 Action orchestration pipelines

And later…

**You can plug in an LLM to rewrite or expand these actions dynamically.**

---

# 9️⃣ **How an Orchestrator Communicates Priority**

The `final_score` is turned into:

* Priority labels
* Severity labels
* Sorted sections
* Executive summaries
* Gap type breakdowns

This teaches:

✔ How to transform numeric priorities into human-readable tiers
✔ How to design executive dashboards
✔ How to justify decisions to humans

If someone asks:

> "Why is this customer in the top 20?"

The report answers it automatically.

---

# 🔟 **This is the "Storytelling Layer" of your agent**

The models do analysis.
The utilities do detection.
The scoring does prioritization.
**The report generation does storytelling.**

Most orchestrators fail not because the logic is bad…
…but because **the output doesn’t make sense to humans.**

This file teaches you how to build:

* Trust
* Clarity
* Consistency
* Actionable next steps
* Business-friendly communication

---

# 🎯 Summary — What You Should Be Learning

### ✔ How to convert structured state → business story

### ✔ How to organize complex insights for executives

### ✔ How to design multi-layered reports

### ✔ How to create reusable communication templates

### ✔ How to build an explainable AI system

### ✔ How to turn detections into actions

### ✔ How to prepare data for LLM rewriting (Phase 8)




In [ ]:
"""
Report Generation Utilities for Revenue Gap Orchestrator

Generate markdown reports with revenue gap analysis and recommendations.
"""

from typing import Dict, List, Any
from datetime import datetime
from pathlib import Path


def generate_revenue_gap_report(
    state: Dict[str, Any]
) -> str:
    """
    Generate comprehensive revenue gap report.

    Args:
        state: Complete orchestrator state

    Returns:
        Markdown report string
    """
    goal = state.get("goal", {})
    gap_summary = state.get("gap_summary", {})
    top_priority_gaps = state.get("top_priority_gaps", [])
    churn_risk_customers = state.get("churn_risk_customers", [])
    customer_revenue_baseline = state.get("customer_revenue_baseline", {})

    # Determine analysis scope
    analysis_scope = goal.get("analysis_scope", "all_customers")
    customer_id = goal.get("customer_id")

    # Build report
    report = f"""# Revenue Gap Analysis Report

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Analysis Scope:** {analysis_scope.replace('_', ' ').title()}
"""

    if customer_id:
        report += f"**Customer ID:** {customer_id}\n"

    report += "\n---\n\n"

    # Executive Summary
    report += "## 📊 Executive Summary\n\n"

    if gap_summary:
        report += f"""
- **Total Customers Analyzed:** {gap_summary.get('total_customers_analyzed', 0)}
- **Customers with Gaps:** {gap_summary.get('customers_with_gaps', 0)}
- **Total Revenue Gap:** ${gap_summary.get('total_revenue_gap', 0.0):,.2f}
- **High Priority Gaps:** {gap_summary.get('high_priority_gaps', 0)}
- **Medium Priority Gaps:** {gap_summary.get('medium_priority_gaps', 0)}
- **Low Priority Gaps:** {gap_summary.get('low_priority_gaps', 0)}
- **Potential Recovery Revenue:** ${gap_summary.get('potential_recovery_revenue', 0.0):,.2f}
"""
    else:
        report += "No gaps detected.\n"

    report += "\n---\n\n"

    # Churn Risk Section
    if churn_risk_customers:
        report += "## ⚠️ Churn Risk Customers\n\n"
        report += f"**Total at Risk:** {len(churn_risk_customers)} customers\n\n"

        # Top 10 churn risks
        top_churn_risks = sorted(
            churn_risk_customers,
            key=lambda x: x.get("churn_risk_score", 0.0),
            reverse=True
        )[:10]

        for i, risk in enumerate(top_churn_risks, 1):
            report += f"""
### {i}. Customer {risk.get('customer_id')}
- **Churn Risk Score:** {risk.get('churn_risk_score', 0.0):.2f}/1.0
- **Weeks Since Last Purchase:** {risk.get('weeks_since_last_purchase', 0)}
- **Risk Factors:** {', '.join(risk.get('risk_factors', []))}
- **Predicted Churn Probability:** {risk.get('predicted_churn_probability', 0.0):.1%}
"""

        report += "\n---\n\n"

    # Top Priority Gaps
    if top_priority_gaps:
        report += "## 🎯 Top Priority Revenue Gaps\n\n"
        report += "### Recommended Action Items\n\n"

        for i, gap in enumerate(top_priority_gaps, 1):
            customer_id = gap.get("customer_id", "Unknown")
            gap_type = gap.get("gap_type", "unknown").replace("_", " ").title()
            gap_amount = gap.get("gap_amount", 0.0)
            gap_percentage = gap.get("gap_percentage", 0.0)
            severity = gap.get("severity", "unknown").upper()
            final_score = gap.get("final_score", 0.0)

            # Get customer details if available
            customer_baseline = customer_revenue_baseline.get(customer_id, {})
            customer_total = customer_baseline.get("total_revenue", 0.0)

            report += f"""
### {i}. Customer {customer_id} - {gap_type}

**Priority Score:** {final_score:.2f}/10.0 | **Severity:** {severity}

**Gap Details:**
- **Gap Amount:** ${abs(gap_amount):,.2f}
- **Gap Percentage:** {gap_percentage:.1f}%
- **Current Revenue:** ${gap.get('current_revenue', 0.0):,.2f}
- **Expected Revenue:** ${gap.get('expected_revenue', 0.0):,.2f}
- **Customer Total Revenue:** ${customer_total:,.2f}

**Scores:**
- Revenue Impact: {gap.get('revenue_impact_score', 0.0):.2f}/10.0
- Churn Risk: {gap.get('churn_risk_score', 0.0):.2f}/10.0
- Customer Value: {gap.get('customer_value_score', 0.0):.2f}/10.0
- Recovery Probability: {gap.get('recovery_probability_score', 0.0):.2f}/10.0

**Rationale:** {gap.get('rationale', 'No rationale provided')}

**Recommended Actions:**
"""

            # Add action recommendations based on gap type
            if gap_type.lower() == "zero spend":
                report += "- Immediate re-engagement campaign\n"
                report += "- Win-back offer with personalized discount\n"
                report += "- Survey to understand reason for inactivity\n"
            elif gap_type.lower() == "declining revenue":
                report += "- Retention campaign with loyalty rewards\n"
                report += "- Upsell/cross-sell opportunities\n"
                report += "- Personalized communication about value\n"
            elif gap_type.lower() == "below baseline":
                report += "- Re-engagement with product recommendations\n"
                report += "- Special offers on frequently purchased items\n"
                report += "- Customer success check-in\n"

            report += "\n"

        report += "---\n\n"

    # Gap Analysis by Type
    if top_priority_gaps:
        report += "## 📈 Gap Analysis by Type\n\n"

        gap_types = {}
        for gap in top_priority_gaps:
            gap_type = gap.get("gap_type", "unknown")
            if gap_type not in gap_types:
                gap_types[gap_type] = []
            gap_types[gap_type].append(gap)

        for gap_type, gaps in gap_types.items():
            type_name = gap_type.replace("_", " ").title()
            total_gap = sum(abs(g.get("gap_amount", 0.0)) for g in gaps)
            avg_score = sum(g.get("final_score", 0.0) for g in gaps) / len(gaps) if gaps else 0.0

            report += f"""
### {type_name}
- **Count:** {len(gaps)} gaps
- **Total Gap Amount:** ${total_gap:,.2f}
- **Average Priority Score:** {avg_score:.2f}/10.0
"""

        report += "\n---\n\n"

    # Recommendations Summary
    report += "## 💡 Strategic Recommendations\n\n"

    if gap_summary:
        total_gap = gap_summary.get("total_revenue_gap", 0.0)
        potential_recovery = gap_summary.get("potential_recovery_revenue", 0.0)
        high_priority = gap_summary.get("high_priority_gaps", 0)

        report += f"""
1. **Immediate Action Required:** Address {high_priority} high-priority gaps to prevent revenue loss
2. **Recovery Opportunity:** ${potential_recovery:,.2f} in potential recovery revenue from top priority customers
3. **Focus Areas:**
   - Prioritize customers with zero spend weeks (highest churn risk)
   - Re-engage declining revenue customers with personalized offers
   - Monitor below-baseline customers for early intervention
"""

    report += "\n---\n\n"

    # Footer
    report += f"""
*Report generated by Revenue Gap Orchestrator*
*Analysis Date: {datetime.now().strftime('%Y-%m-%d')}*
"""

    return report


def save_report(
    report_content: str,
    state: Dict[str, Any],
    reports_dir: str = "output/revenue_gap_reports"
) -> str:
    """
    Save report to file.

    Args:
        report_content: Report markdown content
        state: Orchestrator state
        reports_dir: Directory to save reports

    Returns:
        Path to saved report file
    """
    # Create reports directory
    reports_path = Path(reports_dir)
    reports_path.mkdir(parents=True, exist_ok=True)

    # Generate filename
    goal = state.get("goal", {})
    customer_id = goal.get("customer_id")
    analysis_scope = goal.get("analysis_scope", "all_customers")

    if customer_id:
        filename = f"revenue_gap_report_customer_{customer_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    else:
        filename = f"revenue_gap_report_all_customers_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"

    filepath = reports_path / filename

    # Write report
    with open(filepath, 'w') as f:
        f.write(report_content)

    return str(filepath)



# Report Generation Node

In [ ]:
def report_generation_node(state: PredictiveRevenueGapState) -> Dict[str, Any]:
    """
    Report Generation Node: Generate final revenue gap report.

    Creates a comprehensive markdown report with:
    - Executive summary
    - Top priority gaps
    - Churn risk customers
    - Strategic recommendations
    """
    errors = state.get("errors", [])
    top_priority_gaps = state.get("top_priority_gaps", [])

    if not top_priority_gaps and not state.get("churn_risk_customers"):
        return {
            "errors": errors + ["report_generation_node: No gaps or churn risks to report"]
        }

    # Get reports directory from state or config
    from config import PredictiveRevenueGapConfig
    config = PredictiveRevenueGapConfig()

    reports_dir = state.get("reports_dir") or config.reports_dir

    try:
        # Generate report
        report_content = generate_revenue_gap_report(state)

        # Save report
        report_file_path = save_report(report_content, state, reports_dir=reports_dir)

        return {
            "revenue_gap_report": report_content,
            "report_file_path": report_file_path,
            "errors": errors
        }

    except Exception as e:
        return {
            "errors": errors + [f"report_generation_node: Unexpected error - {str(e)}"]
        }


# Tests for Report Generation Utilities

In [ ]:
"""
Tests for Report Generation Utilities

Testing Phase 6: Report generation utilities before building the node
"""

import sys
from pathlib import Path
from datetime import datetime

# Add project root to path
PROJECT_ROOT = Path(__file__).parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

from agents.revenue_gap_orchestrator.utilities.report_generation import (
    generate_revenue_gap_report,
    save_report
)


def test_generate_revenue_gap_report():
    """Test report generation"""
    state = {
        "goal": {
            "analysis_scope": "all_customers",
            "customer_id": None
        },
        "gap_summary": {
            "total_customers_analyzed": 200,
            "customers_with_gaps": 53,
            "total_revenue_gap": 5000.0,
            "high_priority_gaps": 15,
            "medium_priority_gaps": 20,
            "low_priority_gaps": 18,
            "potential_recovery_revenue": 7500.0
        },
        "top_priority_gaps": [
            {
                "customer_id": "1",
                "gap_type": "declining_revenue",
                "gap_amount": -200.0,
                "gap_percentage": -25.0,
                "severity": "high",
                "current_revenue": 30.0,
                "expected_revenue": 40.0,
                "final_score": 8.5,
                "revenue_impact_score": 7.0,
                "churn_risk_score": 6.5,
                "customer_value_score": 8.0,
                "recovery_probability_score": 7.0,
                "rationale": "Customer revenue declined 25% from baseline"
            }
        ],
        "churn_risk_customers": [
            {
                "customer_id": "2",
                "churn_risk_score": 0.85,
                "weeks_since_last_purchase": 3,
                "risk_factors": ["zero_spend_weeks", "declining_trend"],
                "predicted_churn_probability": 0.75
            }
        ],
        "customer_revenue_baseline": {
            "1": {
                "total_revenue": 400.0
            }
        }
    }

    report = generate_revenue_gap_report(state)

    assert isinstance(report, str)
    assert "# Revenue Gap Analysis Report" in report
    assert "Executive Summary" in report
    assert "Top Priority Revenue Gaps" in report
    assert "Customer 1" in report
    assert "Churn Risk Customers" in report
    assert "Strategic Recommendations" in report
    print("✅ Generate revenue gap report test passed")


def test_generate_revenue_gap_report_single_customer():
    """Test report generation for single customer"""
    state = {
        "goal": {
            "analysis_scope": "single_customer",
            "customer_id": "1"
        },
        "gap_summary": {
            "total_customers_analyzed": 1,
            "customers_with_gaps": 1,
            "total_revenue_gap": 200.0,
            "high_priority_gaps": 1,
            "medium_priority_gaps": 0,
            "low_priority_gaps": 0,
            "potential_recovery_revenue": 200.0
        },
        "top_priority_gaps": [
            {
                "customer_id": "1",
                "gap_type": "declining_revenue",
                "gap_amount": -200.0,
                "gap_percentage": -25.0,
                "severity": "high",
                "final_score": 8.5,
                "rationale": "Customer revenue declined 25% from baseline"
            }
        ],
        "churn_risk_customers": [],
        "customer_revenue_baseline": {}
    }

    report = generate_revenue_gap_report(state)

    assert "Customer ID: 1" in report
    assert "Single Customer" in report
    print("✅ Generate revenue gap report (single customer) test passed")


def test_generate_revenue_gap_report_no_gaps():
    """Test report generation with no gaps"""
    state = {
        "goal": {
            "analysis_scope": "all_customers"
        },
        "gap_summary": {},
        "top_priority_gaps": [],
        "churn_risk_customers": [],
        "customer_revenue_baseline": {}
    }

    report = generate_revenue_gap_report(state)

    assert "No gaps detected" in report
    print("✅ Generate revenue gap report (no gaps) test passed")


def test_save_report():
    """Test saving report to file"""
    report_content = "# Test Report\n\nThis is a test report."
    state = {
        "goal": {
            "analysis_scope": "all_customers",
            "customer_id": None
        }
    }

    # Use a test directory
    test_reports_dir = "output/test_reports"
    filepath = save_report(report_content, state, reports_dir=test_reports_dir)

    assert filepath.endswith(".md")
    assert Path(filepath).exists()

    # Verify content
    with open(filepath, 'r') as f:
        saved_content = f.read()
    assert saved_content == report_content

    # Cleanup
    Path(filepath).unlink()
    print("✅ Save report test passed")


if __name__ == "__main__":
    print("Testing Report Generation Utilities...\n")

    test_generate_revenue_gap_report()
    test_generate_revenue_gap_report_single_customer()
    test_generate_revenue_gap_report_no_gaps()
    test_save_report()

    print("\n✅ All report generation utility tests passed!")



# Test Report Gen Code

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_034_Predictive_Revenue_Gap_Orchestrator %  python3 tests/test_report_generation_utilities.py
Testing Report Generation Utilities...

✅ Generate revenue gap report test passed
✅ Generate revenue gap report (single customer) test passed
✅ Generate revenue gap report (no gaps) test passed
✅ Save report test passed

✅ All report generation utility tests passed!